# Patient 01 Tumor & Tumor-Blood Matching TCR Analysis

In [ ]:
# load packages
import sys
import scanpy as sc
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.cm as cm


In [ ]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set up figure parameters
plt.rcParams['figure.figsize'] = (3.0, 3.0)
sc.settings.verbosity = 0
sc.set_figure_params(dpi=120, dpi_save=600, format='png', figsize=(3, 3))

In [ ]:
# set working and fig dir
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/data/TCR_tumor_blood_analysis/Patient_01"
sc.settings.figdir = os.path.join(work_dir, "plots")
fig_dir = os.path.join(work_dir, "plots")

In [ ]:
# read anndata object
adata = sc.read_h5ad("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/data/outputdata/Patient_01/P01_TNK_TCR_updated_07-05-24.h5ad")
adata

In [ ]:
adata.obs

In [ ]:
adata.obs["Annotation_2.0"].unique()

In [ ]:
# basic T cells annotation
sc.pl.umap(
        adata,
        color="cell_type",
        vmin=0,
        vmax="p99",  
        sort_order=False,  
        frameon=False,
        use_raw=False,
        title="Basic T Cells Annotation",
        legend_fontsize=8,
        save = "_P01_Basic_T_Cells_Annotation.png" 
)

In [ ]:
# plot annotation 2.0
sc.pl.umap(
        adata,
        color="Annotation_2.0",
        vmin=0,
        vmax="p99",  
        sort_order=False,  
        frameon=False,
        use_raw=False,
        title="T Cells Annotation",
        legend_fontsize=8,
        save = "_P01_Refined_T_Cells_Annotation.png"
)

In [ ]:
adata.obs['cloneSize'].unique()

In [ ]:
# order cloneSize var
legend_order = ['Hyperexpanded (50 < X <= 103)', 'Large (20 < X <= 50)', 'Medium (5 < X <= 20)',
               'Small (1 < X <= 5)', 'Single (0 < X <= 1)', 'NA']
adata.obs['cloneSize'] = pd.Categorical(adata.obs['cloneSize'], categories=legend_order)
#adata.obs['cloneSize'] = np.array(adata.obs['cloneSize'])

In [ ]:
color_map = cm.get_cmap('inferno')
colorblind_vector = [color_map(i/4) for i in range(4, -1, -1)]
colorblind_vector.append("grey")
print(colorblind_vector)

In [ ]:
# create a dictionary of cloneSize colors
colorblind_dict = dict(map(lambda i,j : (i,j) , legend_order, colorblind_vector))

In [ ]:
# function to plot multiple umaps next to each other
def split_umap(adata_list, split_by, titles, ncol=2, nrow=1, **kwargs):
    categories = adata.obs[split_by].cat.categories
    if nrow is None:
        nrow = int(np.ceil(len(categories) / ncol))
    fig, axs = plt.subplots(nrow, ncol, figsize=(4*ncol, 3*nrow)) #5,4
    axs = axs.flatten()
    for i, cat in enumerate(categories):
        ax = axs[i]
        if i != len(categories)-1:
            sc.pl.umap(adata_list[i][adata_list[i].obs[split_by] == cat], title=titles[i], ax=ax, show=False,  **kwargs)
        else:
            sc.pl.umap(adata_list[i][adata_list[i].obs[split_by] == cat], title=titles[i], ax=ax, show=False,  **kwargs)

    plt.tight_layout()

In [ ]:
# Plot clonal expansion dynamics of tumor clonotypes 
sc.pl.umap(adata, 
           color=["cloneSize"], 
           palette=colorblind_dict,
           na_color = "grey",
           frameon=False,
           title="Tumor Clonotypes",
           legend_fontsize=8,
           save="_P01_Tumor_Clonotypes_Expansion.png"
)

In [ ]:
# Plot clonal expansion dynamics of blood & tumor matching clonotypes 
sc.pl.umap(adata[adata.obs.blood == "TRUE"],  
           color="cloneSize", 
           palette=colorblind_dict,
           frameon=False,
           title="Tumor-blood Clonotypes",
           legend_fontsize=8,
           save="_P01_Tumor-Blood_Clonotypes_Expansion.png"
)

In [ ]:
adata.obs['timepoint'].unique()

In [ ]:
# order timepoint var

timepoint_order = ['SCR', 'C2']
adata.obs['timepoint'] = pd.Categorical(adata.obs['timepoint'], categories=timepoint_order)

In [ ]:
# Plot clonal expansion dynamics of blood & tumor matching clonotypes split by timepoint
split_umap(adata_list=[adata[adata.obs.blood_pre == "TRUE"], adata[adata.obs.blood_post == "TRUE"]],
           color="cloneSize", 
           split_by="timepoint",
           titles=["T0/-ICI", "T1/+ICI"],
           palette=colorblind_dict,
           frameon=False,
           legend_fontsize=8
          )
plt.savefig(os.path.join(fig_dir, "umap" + "_P01_Tumor-Blood_Clonotypes_Expansion_splitBy_Timepoint.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
split_umap(adata_list=[adata, adata],
           color=["compartment_diffExp"], 
           split_by="timepoint",
           palette=["grey", "green", "red"],
           frameon=False,
           titles=["T0/-ICI", "T1/+ICI"],
           legend_fontsize=8
)
plt.savefig(os.path.join(fig_dir, "umap" + "_P01_diffExp_tumor-blood_clonotypes.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata_filt = adata[adata.obs['compartment_diffExp'] != "NA"].copy()
tmp = pd.crosstab(adata_filt.obs['cell_type'],adata_filt.obs['compartment_diffExp'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(1.6, 1.02),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P01_basic_cell_types_props.png"), dpi=300, format="png", bbox_inches="tight")

In [ ]:
adata_filt = adata[adata.obs['compartment_diffExp'] != "NA"].copy()
tmp = pd.crosstab(adata_filt.obs['Annotation_2.0'], adata_filt.obs['compartment_diffExp'], normalize='columns').T.plot(kind='bar', stacked=True, title="Proportions")
plt.xlabel(None)
tmp.legend(title='Cell Types', bbox_to_anchor=(2.2, 1.2),loc='upper right')
plt.savefig(os.path.join(fig_dir, "P01_adv_cell_types_props.png"), dpi=300, format="png", bbox_inches="tight")